In [ ]:
elastic_rods_dir = '../../elastic_rods/python/'
weaving_dir = '../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

In [ ]:
# Reloadable libraries
import importlib

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

import vis.fields
import matplotlib.cm as cm

In [ ]:
import time

In [ ]:
# Sphere Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 10]
ISO_CS = [4, 4]
MODEL_PATH = osp.join(weaving_dir + 'models/equal_edge_atI.obj')
MODEL_NAME = "equal_edge_atI"
SUBDIVISION_RESOLUTION = 20
REGULARIZATION_WEIGHT = 0.05
SMOOTHING_WEIGHT = 0
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/centered_sphere_100mm.obj')
RIBBON_NAME = "sphere_strip"
HAS_THREE_FAMILIES = False

In [ ]:
# Pseudosphere Small
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [0.005, 0.03]
MODEL_NAME = "small_pseudo_sphere"
MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 20
SMOOTHING_WEIGHT = 10
REGULARIZATION_WEIGHT = 0
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/pseudo_sphere/{}.obj'.format(MODEL_NAME))
RIBBON_NAME = "small_pseudo_sphere_strip"
HAS_THREE_FAMILIES = True

In [ ]:
# Regular Torus Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
default_camera_parameters_2 = ((1.6535518732236205, -2.1434469534967877, -5.518813874877859),
 (-0.5096009366331774, 0.449950326402865, -0.733441973950591),
 (0.1111, 0.1865, 0.5316))
RIBBON_CS = [4, 40]
ISO_CS = [1, 1]
MODEL_PATH = osp.join(weaving_dir + 'models/regular_torus.obj')
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/regular_torus.obj')
MODEL_NAME = 'regular_torus'
SUBDIVISION_RESOLUTION = 20
REGULARIZATION_WEIGHT = 0.5
RIBBON_NAME = 'regular_torus_strip'
HAS_THREE_FAMILIES = True

In [ ]:
# # Freeform 5 
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [15, 150]
# MODEL_NAME = "freeform_5"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 5
# SMOOTHING_WEIGHT = 10
# REGULARIZATION_WEIGHT = 0
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/pseudo_sphere/{}.obj'.format('small_pseudo_sphere'))

# RIBBON_NAME = "freeform_5_strip"

In [ ]:
# # Bird
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [100, 1000]
# MODEL_NAME = "bird"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 20
# SMOOTHING_WEIGHT = 10
# REGULARIZATION_WEIGHT = 0
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/{}.obj'.format(MODEL_NAME))
# RIBBON_NAME = "bird_strip"

In [ ]:
def initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    start_time = time.time()
    l = elastic_rods.SurfaceAttractedLinkage(surface_path, useCenterline, model_path, subdivision_res, False, InterleavingType.weaving)
    construction_time = time.time()
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    set_material_time = time.time()
    l.set_holdClosestPointsFixed(True);
    l.set_attraction_tgt_joint_weight(0.01);
    l.attraction_weight = 100;
    message = ('construct: ', construction_time - start_time, 'set material: ', set_material_time - construction_time)
    return l, message

In [ ]:
with so(): linkage, message = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)

In [ ]:
print(message)

In [ ]:
view = linkage_vis.LinkageViewer(linkage, width=1024, height=640)
view.setCameraParams(default_camera_parameters)
view.show()

In [ ]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 10;
OPTS.beta = 1e-8
OPTS.niter = 100

In [ ]:
joint_normals = np.array([linkage.joint(i).normal for i in range(linkage.numJoints())])
joint_projection_normals = linkage.get_closest_point_normal(linkage.jointPositions())
joint_projection_normals = joint_projection_normals.reshape(joint_normals.shape)
la.norm((joint_projection_normals - joint_normals))

In [ ]:
joint_projection_normals = joint_projection_normals.reshape(joint_normals.shape)

In [ ]:
la.norm((joint_projection_normals - joint_normals))

In [ ]:
start_time = time.time()
with so(): elastic_rods.compute_equilibrium(linkage, options = OPTS)
end_time = time.time()
print(end_time - start_time)

In [ ]:
view.update()

In [ ]:
la.norm(linkage.gradient())

In [ ]:
def get_ribbon_family(linkage):
    # build graph for ribbon family
    import networkx as nx

    ribbons = order_segments_by_ribbons(linkage)

    segment_in_ribbon = -1 * np.ones(linkage.numSegments())
    for ri in range(len(ribbons)):
        for (seg_index, _) in ribbons[ri]:
            segment_in_ribbon[seg_index] = ri

    # print(segment_in_ribbon)
    G = nx.Graph()
    G.add_nodes_from(range(len(ribbons)))
    for ji in range(linkage.numJoints()):
        seg_1 = linkage.joint(ji).segments_A[0]
        seg_2 = linkage.joint(ji).segments_B[0]
        ribbon_1 = int(segment_in_ribbon[seg_1])
        ribbon_2 = int(segment_in_ribbon[seg_2])
        G.add_edge(ribbon_1, ribbon_2)

    ribbon_family = list(-1 * np.ones(len(ribbons)))
    ribbon_family[0] = 'A'
    neighbor = [n for n in G[0]][0]
    ribbon_family[neighbor] = 'B'

    C_family = sorted(nx.common_neighbors(G, 0, neighbor))
    for index in C_family:
        ribbon_family[index] = 'C'

    B_family = sorted(nx.common_neighbors(G, 0, C_family[0]))
    for index in B_family:
        ribbon_family[index] = 'B'

    A_family = sorted(nx.common_neighbors(G, B_family[0], C_family[0]))
    for index in A_family:
        ribbon_family[index] = 'A'

    for a_e in A_family:
        for b_e in B_family:
            C_family.extend(sorted(nx.common_neighbors(G, a_e, b_e)))
            C_family = list(set(C_family))
    for index in C_family:
        ribbon_family[index] = 'C'

    for a_e in A_family:
        for c_e in C_family:
            B_family.extend(sorted(nx.common_neighbors(G, a_e, c_e)))
            B_family = list(set(B_family))
    for index in B_family:
        ribbon_family[index] = 'B'

    for b_e in B_family:
        for c_e in C_family:
            A_family.extend(sorted(nx.common_neighbors(G, b_e, c_e)))
            A_family = list(set(A_family))
    for index in A_family:
        ribbon_family[index] = 'A'
    assert(len(A_family) + len(B_family) + len(C_family) == len(ribbons))
    return ribbon_family

In [ ]:
def write_linkage_ribbon_output_florin(linkage, use_family_label = False):
    ribbons = order_segments_by_ribbons(linkage)
    
    ribbon_centerline_index = []
    ribbon_centerline_points = []
    ribbon_centerline_normal = []

    ribbon_loop_indicator = []
    curr_offset = 0

    ribbon_family = range(len(ribbons))
    if use_family_label:
        ribbon_family = get_ribbon_family(linkage)

    def normalize(vec):
        return vec/la.norm(vec)

    def is_valid_index(index):
        return index < linkage.numSegments()

    for ribbon in ribbons:
        print(ribbon)
        curr_ribbon_centerline_index = []

        curr_start_joint = linkage.segment(ribbon[0][0]).startJoint
        prev_segment = linkage.joint(curr_start_joint).continuationSegment(ribbon[0][0])
        if is_valid_index(prev_segment):
            ribbon_loop_indicator.append('L')
        else:
            ribbon_loop_indicator.append('O')

        for (segment_index, orientation) in ribbon:
            if orientation != 1:
                print('Rod orientation incorrect!')
            curr_rod = linkage.segment(segment_index).rod
            dc = curr_rod.deformedConfiguration()
            curr_start_joint = linkage.segment(segment_index).startJoint
            curr_end_joint = linkage.segment(segment_index).endJoint

            prev_segment = linkage.joint(curr_start_joint).continuationSegment(segment_index)
            next_segment = linkage.joint(curr_end_joint).continuationSegment(segment_index)
            
            count_point = 0
            if not is_valid_index(prev_segment):
                ribbon_centerline_points.append(curr_rod.deformedPoints()[0])
                ribbon_centerline_normal.append(dc.materialFrame[0].d2)
                count_point += 1

            ribbon_centerline_points.extend(curr_rod.deformedPoints()[1:-1])
            ribbon_centerline_normal.extend([normalize(dc.materialFrame[edge_index - 1].d2 + dc.materialFrame[edge_index].d2) for edge_index in range(SUBDIVISION_RESOLUTION)[1:]])
            count_point += SUBDIVISION_RESOLUTION - 1
            
            if not is_valid_index(next_segment):
                ribbon_centerline_points.append(curr_rod.deformedPoints()[-1])
                ribbon_centerline_normal.append(dc.materialFrame[-1].d2)
                count_point += 1

            curr_centerline_index = np.arange(curr_offset, curr_offset + count_point)
            curr_offset += count_point
            
            curr_ribbon_centerline_index.extend(curr_centerline_index)
        ribbon_centerline_index.append(curr_ribbon_centerline_index)
    print(len(ribbon_centerline_index))
    print(len(ribbon_centerline_normal))
    print(len(ribbon_centerline_points))

    if not os.path.exists(RIBBON_NAME):
        os.makedirs(RIBBON_NAME)

    with open('{}/{}_polylines.txt'.format(RIBBON_NAME, RIBBON_NAME), 'w') as f:
        ribbon_count = 0
        for line in ribbon_centerline_index:
            f.write('{} {} {}\n'.format(ribbon_family[ribbon_count], ribbon_loop_indicator[ribbon_count], ' '.join([str(x) for x in line])))
            ribbon_count += 1

    with open('{}/{}_points.txt'.format(RIBBON_NAME, RIBBON_NAME), 'w') as f:
        for point in ribbon_centerline_points:
            f.write('{}\n'.format(' '.join([str(x) for x in list(point)])))

    with open('{}/{}_normals.txt'.format(RIBBON_NAME, RIBBON_NAME), 'w') as f:
        for normal in ribbon_centerline_normal:
            f.write('{}\n'.format(' '.join([str(x) for x in list(normal)])))

    with open('{}/{}.obj'.format(RIBBON_NAME, RIBBON_NAME), 'w') as f:
        for point in ribbon_centerline_points:
            f.write('v {}\n'.format(' '.join([str(x) for x in list(point)])))

        for line in ribbon_centerline_index:
            for i in range(len(line) - 1):
                f.write('l {} {}\n'.format(line[i]+1, line[i+1]+1))


In [ ]:
write_linkage_ribbon_output_florin(linkage, HAS_THREE_FAMILIES)

In [ ]:
def highlight_rod_and_joint(linkage, strip_index, select_joint_index):
    ribbons = order_segments_by_ribbons(linkage)
    new_rod, fixedVars = construct_elastic_rod_loop_from_rod_segments(linkage, ribbons[strip_index])
    # Set the material of the new rod to be the same as previously.
    new_rod.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
    single_rod_view_compare = linkage_vis.LinkageViewer(linkage, width=1024, height=640)
    single_rod_view = linkage_vis.LinkageViewer(new_rod, width=1024, height=640)

    j = linkage.joint(select_joint_index)
    seg_index = j.segments_A[0]
    vx_index = 0
    if linkage.segment(seg_index).startJoint != select_joint_index:
        vx_index = -1
#     joint_vector_field = [np.zeros_like(np.reshape(s.rod.gradient()[0:3*s.rod.numVertices()], (s.rod.numVertices(), 3))) for s in linkage.segments()]
#     joint_vector_field[seg_index][vx_index] = linkage.segment(seg_index).rod.deformedConfiguration().materialFrame[vx_index].d2
#     single_rod_view_compare.update(vectorField=joint_vector_field)

    sf = vis.fields.ScalarField(new_rod, 0.6 * np.ones_like(np.array(new_rod.deformedConfiguration().len)), colormap=cm.Blues, vmin = 0, vmax = 1)

    single_rod_view_compare.update(mesh = single_rod_view.mesh, preserveExisting=True, scalarField=sf)
    single_rod_view_compare.setCameraParams(((0.1380416750325228, 0.9648987923360046, 4.776431269112697),
     (0.9983340296894934, -0.054896765875897646, -0.01776260848808606),
     (0.0, 0.0, 0.0)))
    return single_rod_view_compare

In [ ]:
# single_rod_view_compare = highlight_rod_and_joint(straight_linkage, 0, 0)

In [ ]:
ribbons = order_segments_by_ribbons(linkage)
all_centerline_pos, all_rest_kappas, all_material_frame = concatenate_rod_properties_from_rod_segments(linkage, ribbons[0])
# Set the centerline pos and initialize thetas to zero.
new_rod = elastic_rods.ElasticRod(all_centerline_pos)
# Set the rest kappa to be the same as the rod strip in the linkage.
new_rod.setRestKappas(all_rest_kappas)
# Set the reference frame equal to the extracted deformed curve's material frame (and leave thetas at zero).
for i in range(len(all_material_frame)):
    new_rod.deformedConfiguration().referenceDirectors[i].d1 = all_material_frame[i].d1
    new_rod.deformedConfiguration().referenceDirectors[i].d2 = all_material_frame[i].d2
num_material_frame = len(all_material_frame)
new_rod.deformedConfiguration().referenceDirectors[num_material_frame - 1].d1 = - all_material_frame[num_material_frame - 1].d1
new_rod.deformedConfiguration().referenceDirectors[num_material_frame - 1].d2 = - all_material_frame[num_material_frame - 1].d2

new_rod.updateSourceFrame()
new_rod.setDeformedConfiguration(all_centerline_pos, np.zeros(len(all_material_frame)))

# If the rod is a loop, then we need to fixed the end edges of the rod by fixing four vertex positions and two thetas.
last_two_pos = (len(all_centerline_pos) - 2) * 3
fixedPositionVars = list(range(6)) + list(range(last_two_pos, last_two_pos + 6)) 
fixedThetaVars = list([x + len(all_centerline_pos) * 3 for x in [0, len(all_material_frame) - 1]])
fixedVars = fixedPositionVars + fixedThetaVars

In [ ]:
new_rod.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
with so(): elastic_rods.compute_equilibrium(new_rod, options = OPTS, fixedVars = fixedVars)
mobius_view = linkage_vis.LinkageViewer(new_rod, width=1024, height=640)
mobius_view.setCameraParams(default_camera_parameters)

In [ ]:
mobius_view.show()

In [ ]:
single_rod_view_compare.setCameraParams(default_camera_parameters)
single_rod_view_compare.show()

In [ ]:
# j = straight_linkage.joint(44)
# seg_index = j.segments_A[0]
# vx_index = 0
# if straight_linkage.segment(seg_index).startJoint != 44:
#     vx_index = -1
# joint_vector_field = [np.zeros_like(np.reshape(s.rod.gradient()[0:3*s.rod.numVertices()], (s.rod.numVertices(), 3))) for s in straight_linkage.segments()]
# joint_vector_field[seg_index][vx_index] = straight_linkage.segment(seg_index).rod.deformedConfiguration().materialFrame[vx_index].d2
# view.update()

In [ ]:
# normals = []
# joint_indices = []
# with open('bird_joint_normal_info.txt', 'r') as f:
#     for line in f.readlines():
#         if 'normals' in line:
#             normals.append(np.array([float(x) for x in line.strip().split(' ')[2:]]))
#         if 'index' in line:
#             joint_indices.append(int(line.strip().split(' ')[3]))

In [ ]:
# joint_vector_field = [np.zeros_like(np.reshape(s.rod.gradient()[0:3*s.rod.numVertices()], (s.rod.numVertices(), 3))) for s in straight_linkage.segments()]
# for i in range(len(normals)):
#     j = straight_linkage.joint(joint_indices[i])
#     seg_index = j.segments_A[0]
#     vx_index = 0
#     if straight_linkage.segment(seg_index).startJoint != joint_indices[i]:
#         vx_index = -1
#     joint_vector_field[seg_index][vx_index] = normals[i]
    

In [ ]:
# view.update(vectorField=joint_vector_field)